In [ ]:
print('Gajraj')

In [ ]:
# !pip install faiss-cpu sentence-transformers
# !pip install pandas numpy matplotlib
# %%capture
# !pip install git+https://github.com/neuml/txtai

In [ ]:
import pandas as pd
data = [['Where are your headquarters located?', 'location'],
['Throw my cellphone in the water', 'random'],
['Network Access Control?', 'networking'],
['Address', 'location']]
df = pd.DataFrame(data, columns = ['text', 'category'])

In [ ]:
from sentence_transformers import SentenceTransformer
text = df['text']
encoder = SentenceTransformer("paraphrase-mpnet-base-v2")
vectors = encoder.encode(text)

In [ ]:
import faiss

vector_dimension = vectors.shape[1]
index = faiss.IndexFlatL2(vector_dimension)
faiss.normalize_L2(vectors)
index.add(vectors)

In [ ]:
import numpy as np

search_text = 'where is your office?'
search_vector = encoder.encode(search_text)
_vector = np.array([search_vector])
faiss.normalize_L2(_vector)

In [ ]:
k = index.ntotal
distances, ann = index.search(_vector, k=k)

In [ ]:
results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})

In [ ]:
# join by: df1.ann == data.index 
merge = pd.merge(results, df, left_on='ann', right_index=True)
merge

In [ ]:
labels  = df['category']
category = labels[ann[0][0]]
category